# 패키지 업데이트

In [4]:
!pip install -U trl bitsandbytes -q
!pip install -U bitsandbytes -q
!pip install peft -q

# 환경설정

In [15]:
import os
os.environ["CUDA_VISIBLE_DEVICES"]="0"
os.environ["WANDB_MODE"] = "disabled"

In [2]:
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig
from peft import LoraConfig, get_peft_model
from trl import SFTTrainer, SFTConfig
from datasets import load_dataset

/home/user/jhyuk/venvs/.gpt_oss_vllm/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


# 모델 불러오기

In [4]:
model_name = "KORMo-Team/KORMo-sft"

# 4bit 양자화 설정
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16
)

# 모델 로드 (4bit 양자화)
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    quantization_config=bnb_config,
    device_map="auto",
    trust_remote_code=True
)

# 토크나이저 로드
tokenizer = AutoTokenizer.from_pretrained(
    model_name,
    trust_remote_code=True
)

print(f"✓ 모델 로드 완료: {model_name}")

Loading checkpoint shards: 100%|██████████| 5/5 [00:22<00:00,  4.43s/it]


✓ 모델 로드 완료: KORMo-Team/KORMo-sft


# 모델 추론

In [5]:
test_prompt = "영희가 연필 12개를 가지고 있었는데 철수가 절반을 가져가고 영수가 공책 3개를 가져갔으면 영희에게 남은 연필의 갯수는 몇개인가요?"

In [6]:
# Think 추론
print("\n[Think 추론]")
messages_think = [{"role": "user", "content": test_prompt}]
input_text_think = tokenizer.apply_chat_template(messages_think, tokenize=False, add_generation_prompt=True, enable_thinking=True)
inputs_think = tokenizer(input_text_think, return_tensors="pt").to(model.device)

with torch.no_grad():
    outputs_think = model.generate(
        **inputs_think,
        max_new_tokens=1024,
        do_sample=True,
        temperature=0.7,
        pad_token_id=tokenizer.eos_token_id
    )
response_think = tokenizer.decode(outputs_think[0], skip_special_tokens=True)
print(f"입력: {test_prompt}")
print(f"출력: {response_think}")


[Think 추론]
입력: 영희가 연필 12개를 가지고 있었는데 철수가 절반을 가져가고 영수가 공책 3개를 가져갔으면 영희에게 남은 연필의 갯수는 몇개인가요?
출력: user
영희가 연필 12개를 가지고 있었는데 철수가 절반을 가져가고 영수가 공책 3개를 가져갔으면 영희에게 남은 연필의 갯수는 몇개인가요?
assistant
<think>
영희가 처음에 가지고 있던 연필의 수는 12개입니다.  
철수가 이 중 절반을 가져갔으므로, 철수가 가져간 연필의 수는 12 ÷ 2 = 6개입니다.  
따라서 철수가 가져간 후 영희에게 남은 연필의 수는 12 - 6 = 6개입니다.  
이후 영수가 공책 3개를 가져갔으나, 공책은 연필이 아니므로 영희에게 남은 연필의 수에는 영향을 주지 않습니다.  
따라서 영희에게 남은 연필의 수는 6개입니다.
</think>

영희가 처음에 가진 연필은 총 12개입니다.  
철수가 이 중 절반을 가져갔으므로, 철수가 가져간 연필 수는  
$ 12 \div 2 = 6 $개입니다.  

철수가 가져간 후 남은 연필의 수는  
$ 12 - 6 = 6 $개입니다.  

영수가 공책 3개를 가져갔지만, 공책은 연필이 아니므로  
영희에게 남은 연필의 수에는 영향을 주지 않습니다.  

따라서, 영희에게 남은 연필의 수는 **6개**입니다.


In [7]:
# Non-think 추론
print("\n[Non-think 추론]")
messages = [{"role": "user", "content": test_prompt}]
input_text = tokenizer.apply_chat_template(messages, tokenize=False, add_generation_prompt=True, enable_thinking=False)
inputs = tokenizer(input_text, return_tensors="pt").to(model.device)

with torch.no_grad():
    outputs = model.generate(
        **inputs,
        max_new_tokens=1024,
        do_sample=False,
        pad_token_id=tokenizer.eos_token_id
    )
response = tokenizer.decode(outputs[0], skip_special_tokens=True)
print(f"입력: {test_prompt}")
print(f"출력: {response}")




[Non-think 추론]
입력: 영희가 연필 12개를 가지고 있었는데 철수가 절반을 가져가고 영수가 공책 3개를 가져갔으면 영희에게 남은 연필의 갯수는 몇개인가요?
출력: user
영희가 연필 12개를 가지고 있었는데 철수가 절반을 가져가고 영수가 공책 3개를 가져갔으면 영희에게 남은 연필의 갯수는 몇개인가요?
assistant
<think>

</think>
영희가 처음에 **연필 12개**를 가지고 있었습니다.
철수가 **절반**을 가져갔으므로:

\[
\frac{1}{2} \times 12 = 6 \text{개}
\]

따라서 철수가 6개를 가져가고, 영희에게 남은 연필은:

\[
12 - 6 = 6 \text{개}
\]

그 후, 영수가 **공책 3개**를 가져갔다고 했는데, 문제는 **연필의 개수**를 묻고 있습니다.  
공책은 연필과 관련이 없으므로, **영희의 연필 수에는 영향을 주지 않습니다**.

따라서, 영희에게 남은 연필의 개수는:

\[
\boxed{6}
\]


# KORMo SFT 데이터 일부 불러오기

In [14]:
dataset = load_dataset(
    'KORMo-Team/KORMo-tutorial-datasets',
    name='sft',
    split='train[11000:15000]'
).rename_columns({'conversation': 'messages'})

print(f"✓ 데이터셋 로드 완료: {len(dataset)}개 샘플")
print(f"✓ 데이터셋 컬럼: {dataset.column_names}")
print(f"\n샘플 예시:")
print(dataset[5])
print(dataset[2999])

✓ 데이터셋 로드 완료: 4000개 샘플
✓ 데이터셋 컬럼: ['messages']

샘플 예시:
{'messages': [{'content': 'Solve the following math problem. Make sure to put the answer (and only answer) inside \\boxed{}.\n\nFind the number of ascending integers which are less than $10^9$. An ascending integer is defined as a positive integer $d$ where in its decimal representation $d=d_md_{m-1}\\ldots d_2d_1$, we have $0 < d_m \\le d_{m-1} \\le \\ldots \\le d_2 \\le d_1$.', 'reasoning_content': None, 'role': 'user'}, {'content': 'To solve the problem of finding the number of **ascending integers less than $10^9$**, we begin by understanding the definition of an ascending integer. It is a **positive integer** where the digits are **non-decreasing** from left to right, and **no digit is zero**. That is, in its decimal representation $d = d_md_{m-1} \\ldots d_1$, we have:\n\n$$\n0 < d_m \\le d_{m-1} \\le \\ldots \\le d_1\n$$\n\nThis implies that:\n- All digits are from the set $\\{1, 2, \\ldots, 9\\}$,\n- The digits are in non-d

# 모델 학습

In [18]:
# LoRA 설정
peft_config = LoraConfig(
    r=128,
    lora_alpha=256,
    lora_dropout=0.05,
    bias="none",
    task_type="CAUSAL_LM",
    target_modules="all-linear"
)

# 학습 인자 설정
training_args = SFTConfig(
    output_dir="./KORMo-IFT-step-qlora-sft",
    num_train_epochs=3,
    per_device_train_batch_size=1,
    gradient_accumulation_steps=2,
    gradient_checkpointing=True,
    optim="adamw_bnb_8bit",
    logging_steps=10,
    save_strategy="epoch",
    learning_rate=2e-4,
    bf16=True,
    warmup_ratio=0.01,
    lr_scheduler_type="cosine",
    packing=True,
    #max_seq_length=1024,
    report_to=None
)

# SFT Trainer 초기화
trainer = SFTTrainer(
    model=model,
    train_dataset=dataset,
    peft_config=peft_config,
    args=training_args,
)

The repository KORMo-Team/KORMo-sft contains custom code which must be executed to correctly load the model. You can inspect the repository content at https://hf.co/KORMo-Team/KORMo-sft .
 You can inspect the repository content at https://hf.co/KORMo-Team/KORMo-sft.
You can avoid this prompt in future by passing the argument `trust_remote_code=True`.

Do you wish to run the custom code? [y/N] y


Tokenizing train dataset:   0%|          | 0/1000 [00:00<?, ? examples/s]

Packing train dataset:   0%|          | 0/1000 [00:00<?, ? examples/s]

In [19]:
# 학습 시작
print("✓ 학습 시작...")
trainer.train()

# 모델 저장
print("\n✓ 학습 완료! 모델 저장 중...")
trainer.model.save_pretrained("./KORMo-IFT-step-qlora-sft")
tokenizer.save_pretrained("./KORMo-IFT-step-qlora-sft")
print("✓ 모델 저장 완료: ./KORMo-IFT-step-qlora-sft")

✓ 학습 시작...


/usr/local/lib/python3.12/dist-packages/notebook/notebookapp.py:191: SyntaxWarning: invalid escape sequence '\/'
  | |_| | '_ \/ _` / _` |  _/ -_)
/usr/local/lib/python3.12/dist-packages/torch/_dynamo/eval_frame.py:929: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. In version 2.5 we will raise an exception if use_reentrant is not passed. use_reentrant=False is recommended, but if you need to preserve the current default behavior, you can pass use_reentrant=True. Refer to docs for more details on the differences between the two variants.
  return fn(*args, **kwargs)


Step,Training Loss


KeyboardInterrupt: 

In [16]:
# Non-think 추론
print("\n[Non-think 추론]")
messages = [{"role": "user", "content": test_prompt}]
input_text = tokenizer.apply_chat_template(messages, tokenize=False, add_generation_prompt=True, enable_thinking=False)
inputs = tokenizer(input_text, return_tensors="pt").to(model.device)

with torch.no_grad():
    outputs = model.generate(
        **inputs,
        max_new_tokens=1024,
        do_sample=False,
        pad_token_id=tokenizer.eos_token_id
    )
response = tokenizer.decode(outputs[0], skip_special_tokens=True)
print(f"입력: {test_prompt}")
print(f"출력: {response}")




[Non-think 추론]
입력: 영희가 연필 12개를 가지고 있었는데 철수가 절반을 가져가고 영수가 공책 3개를 가져갔으면 영희에게 남은 연필의 갯수는 몇개인가요?
출력: user
영희가 연필 12개를 가지고 있었는데 철수가 절반을 가져가고 영수가 공책 3개를 가져갔으면 영희에게 남은 연필의 갯수는 몇개인가요?
assistant
<think>

</think>
영희가 처음에 **연필 12개**를 가지고 있었습니다.
철수가 **절반**을 가져갔으므로:

\[
\frac{1}{2} \times 12 = 6 \text{개}
\]

따라서 철수가 6개를 가져가고, 영희에게 남은 연필은:

\[
12 - 6 = 6 \text{개}
\]

그 후, 영수가 **공책 3개**를 가져갔다고 했는데, 문제는 **연필의 개수**를 묻고 있습니다.  
공책은 연필과 관련이 없으므로, **영희의 연필 수에는 영향을 주지 않습니다**.

따라서, 영희에게 남은 연필의 개수는:

\[
\boxed{6}
\]
